* 测试moss关于输入序列长度和GPU显存的关系

In [30]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt 
import numpy as np
import os

In [31]:
x, y =[], []
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# model = AutoModelForCausalLM.from_pretrained("fnlp/moss-moon-003-sft-int4", trust_remote_code=True).half().cuda("cuda:1")
model = AutoModelForCausalLM.from_pretrained("../model/moss-moon-003-sft-int4", trust_remote_code=True).half().cuda("cuda:0")

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_26325/138497740.py:4 in <module>                                                  │
│                                                                                                  │
│   1 x, y =[], []                                                                                 │
│   2 os.environ["CUDA_LAUNCH_BLOCKING"]="1"                                                       │
│   3 # model = AutoModelForCausalLM.from_pretrained("fnlp/moss-moon-003-sft-int4", trust_remo     │
│ ❱ 4 model = AutoModelForCausalLM.from_pretrained("../model/moss-moon-003-sft-int4", trust_re     │
│   5                                                                                              │
│                                                                                                  │
│ /home/qs/anaconda3/envs/moss/lib/python3.8/site-packages/torch/nn/modules/module.py:749 in cuda  │
│                                                                                                  │
│    746 │   │   Returns:                                                                          │
│    747 │   │   │   Module: self                                                                  │
│    748 │   │   """                                                                               │
│ ❱  749 │   │   return self._apply(lambda t: t.cuda(device))                                      │
│    750 │                                                                                         │
│    751 │   def ipu(self: T, device: Optional[Union[int, device]] = None) -> T:                   │
│    752 │   │   r"""Moves all model parameters and buffers to the IPU.                            │
│                                                                                                  │
│ /home/qs/anaconda3/envs/moss/lib/python3.8/site-packages/torch/nn/modules/module.py:641 in       │
│ _apply                                                                                           │
│                                                                                                  │
│    638 │                                                                                         │
│    639 │   def _apply(self, fn):                                                                 │
│    640 │   │   for module in self.children():                                                    │
│ ❱  641 │   │   │   module._apply(fn)                                                             │
│    642 │   │                                                                                     │
│    643 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    644 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /home/qs/anaconda3/envs/moss/lib/python3.8/site-packages/torch/nn/modules/module.py:641 in       │
│ _apply                                                                                           │
│                                                                                                  │
│    638 │                                                                                         │
│    639 │   def _apply(self, fn):                                                                 │
│    640 │   │   for module in self.children():                                                    │
│ ❱  641 │   │   │   module._apply(fn)                                                             │
│    642 │   │                                                                                     │
│    643 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    644 │   │   │   if torch._has_compatible_shallow_copy_ty

In [ ]:
for i in range(64, 2500, 32):
    demo_data = torch.zeros((1,i),dtype=torch.long).to("cuda:1")
    _=model(demo_data) 
    x.append(i)
    y.append(torch.cuda.memory_allocated('cuda:1')/1024/1024)

In [ ]:
plt.plot(x,y)
coeff = np.polyfit(x,y,2) 
p=np.poly1d(coeff) 
plt.plot(x, p(x)) 
print(p)